In [ ]:
import pyvista as pv
import vtk
import numpy as np

from trame.app import get_server
from pyvista.trame.ui import plotter_ui
from trame.ui.vuetify3 import SinglePageWithDrawerLayout
from trame.widgets import vuetify3
from pyvista.plotting.themes import DocumentTheme # Creating a theme

In [ ]:
# -----------------------------------------------------------------------------
# Constants
# -----------------------------------------------------------------------------
class Representation:
    """
    Constants for different types of representations of VTK actors.

    Attributes:
        Points (int): Representation as points.
        Wireframe (int): Representation as wireframe.
        Surface (int): Representation as a surface.
        SurfaceWithEdges (int): Representation as a surface with edges.
    """
    Points = 0
    Wireframe = 1
    Surface = 2
    SurfaceWithEdges = 3

class LookupTable:
    """
    Constants for different types of lookup tables for color maps.

    Attributes:
        Rainbow (int): Rainbow color map.
        Inverted_Rainbow (int): Inverted rainbow color map.
        Greyscale (int): Greyscale color map.
        Inverted_Greyscale (int): Inverted greyscale color map.
    """
    Rainbow = 0
    Inverted_Rainbow = 1
    Greyscale = 2
    Inverted_Greyscale = 3

In [ ]:
# Server Trame Setup
server = get_server()
state, ctrl = server.state, server.controller

In [ ]:
# Theme testing
my_theme = DocumentTheme()
my_theme.background = '#dddddd'
# my_theme.show_vertices = True
my_theme.show_edges = True
my_theme.split_sharp_edges = True
my_theme.edge_color = 'k'
my_theme.enable_camera_orientation_widget = True # Creates the camera bars in TOP RIGHT
pv.global_theme.load_theme(my_theme)

# Hardcode the backend ('trame' = remote & local, 'client' = local)
# pv.set_jupyter_backend('client')

In [ ]:
# Read the VTK file using PyVista
filename = "file.vtk"
mesh = pv.read(filename)

In [ ]:
# Create the plotter
plotter = pv.Plotter(notebook=True)

# Set up the camera and view
plotter.view_xy()
plotter.add_axes()
plotter.show_grid()

In [ ]:
# Extract data arrays
dataset_arrays = []
point_data = mesh.point_data
cell_data = mesh.cell_data

# Extract point data
for i, (name, array) in enumerate(point_data.items()):
    array_range = np.min(array), np.max(array)
    dataset_arrays.append(
        {
            "text": name,
            "value": i,
            "range": list(array_range),
            "type": vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS,
        }
    )

# Extract cell data
for i, (name, array) in enumerate(cell_data.items()):
    array_range = np.min(array), np.max(array)
    dataset_arrays.append(
        {
            "text": name,
            "value": i,
            "range": list(array_range),
            "type": vtk.vtkDataObject.FIELD_ASSOCIATION_CELLS,
        }
    )

default_array = dataset_arrays[0]
default_min, default_max = default_array.get("range")

In [ ]:
# Add the default mesh
actor = plotter.add_mesh(mesh, scalars=default_array.get("text"), cmap="rainbow")
mapper = actor.mapper

In [ ]:
# Representation Setup and Callbacks

# The callback to change how the VTK displays when an option is selected
def update_representation(actor, mode):
    """
    Update the representation mode of an actor.

    Args:
        actor (vtk.vtkActor): The VTK actor to update.
        mode (int): The representation mode (Points, Wireframe, Surface, SurfaceWithEdges).
    """
    property = actor.GetProperty()
    if mode == Representation.Points:
        property.SetRepresentationToPoints()
        property.SetPointSize(5)
        property.EdgeVisibilityOff()
    elif mode == Representation.Wireframe:
        property.SetRepresentationToWireframe()
        property.SetPointSize(1)
        property.EdgeVisibilityOff()
    elif mode == Representation.Surface:
        property.SetRepresentationToSurface()
        property.SetPointSize(1)
        property.EdgeVisibilityOff()
    elif mode == Representation.SurfaceWithEdges:
        property.SetRepresentationToSurface()
        property.SetPointSize(1)
        property.EdgeVisibilityOn()
    
# Event Listener for changing Representation
@state.change("mesh_representation")
def update_mesh_representation(mesh_representation, **kwargs):
    """
    State change callback to update the representation mode of the mesh.

    Args:
        mesh_representation (int): The new representation mode.
    """
    update_representation(actor, mesh_representation)
    ctrl.view_update()

# Set the initial state to default to on load
state.mesh_representation = Representation.SurfaceWithEdges
update_representation(actor, state.mesh_representation)

In [ ]:
# Topbar of Buttons for GUI
def standard_buttons():
    """
    Define standard buttons for the GUI, including a checkbox for dark mode and a button to reset the camera.
    """
    vuetify3.VCheckbox(
        v_model="$vuetify.theme.dark",
        on_icon="mdi-lightbulb-off-outline",
        off_icon="mdi-lightbulb-outline",
        classes="mx-1",
        hide_details=True,
        dense=True,
    )
    with vuetify3.VBtn(icon=True, click="$refs.view.reset_camera()"):
        vuetify3.VIcon("mdi-crop-free")


In [ ]:
# Drawer Card to house all UI elements

def drawer_card(title):
    """
    Create a UI card component for organizing GUI elements.

    Args:
        title (str): The title of the card.
        ui_name (str): The name used to show/hide the card based on the active UI state.

    Returns:
        vuetify3.VCardText: The content area of the card.
    """
    with vuetify3.VCard():
        vuetify3.VCardTitle(
            title,
            classes="grey lighten-1 py-1 grey--text text--darken-3",
            style="user-select: none; cursor: pointer",
            hide_details=True,
            dense=True,
        )
        content = vuetify3.VCardText(classes="py-2")
    return content

def representation_dropdown():
    """
    The dropdown UI for selecting different representations, e.g. including edges, wireframe, points, etc.
    """
    vuetify3.VSelect(
        v_model=("mesh_representation", state.mesh_representation),
        items=(
            'representations',
            [
                {"title": "Points", "value": Representation.Points},
                {"title": "Wireframe", "value": Representation.Wireframe},
                {"title": "Surface", "value": Representation.Surface},
                {"title": "SurfaceWithEdges", "value": Representation.SurfaceWithEdges},
            ],
        ),
        label="Representation",
        hide_details=True,
        dense=True,
        outlined=True,
        classes="pt-1",
    )

In [ ]:
# TODO: Get ZLayer Callbacks working
def update_zlayer(z_value, actor, **kwargs):
    # Replace actor with z_layer
    plotter.remove_actor(actor)

    z_layer = mesh.threshold([z_value, default_max], scalars='tentlevel')
    actor = plotter.add_mesh(z_layer, scalars=default_array.get("text"), cmap="rainbow")

    plotter.render()

    return actor

@state.change("z_value")
def update_zvalue(z_value, **kwargs):
    global actor
    actor = update_zlayer(z_value, actor)

def level_slider():
    vuetify3.VSlider(
        v_model=("z_value", 0),
        min=int(default_min),
        max=int(default_max),
        step=1,
        label="Level",
        classes="mt-1",
        hide_details=True,
        dense=True,
        thumb_label=True
    )

In [ ]:
# Placeholder for Color Map UI
def test_table():
    with vuetify3.VRow(classes="pt-2", dense=True):
        with vuetify3.VCol(cols="6"):
            vuetify3.VCardTitle(
                "Default",
                classes="grey lighten-1 py-1 grey--text text--darken-3",
                style="user-select: none; cursor: pointer",
                hide_details=True,
                dense=True,
            )
        with vuetify3.VCol(cols="6"):
            vuetify3.VCardTitle(
                "Default",
                classes="grey lighten-1 py-1 grey--text text--darken-3",
                style="user-select: none; cursor: pointer",
                hide_details=True,
                dense=True,
            )

In [ ]:
# WebGUI to Draw and Render VTK + UI

with SinglePageWithDrawerLayout(server) as layout:
    layout.title.set_text("Spacetime Tents Visualization")

    # Top Toolbar Components
    with layout.toolbar:
        vuetify3.VSpacer()
        vuetify3.VDivider(vertical=True, classes="mx-2")
        standard_buttons()

    # Side Drawer Components
    with layout.drawer as drawer:
        drawer.width = 325
        vuetify3.VDivider(classes="mb-2")

        with drawer_card(title="Tents"):
            # Representation Dropdown
            representation_dropdown()

            # Temporary Test (Slot-in for Color Map)
            test_table()

            # Level Slider
            level_slider()
        
    with layout.content:
        # content components
        with vuetify3.VContainer(
            fluid=True,
            classes="pa-0 fill-height",
        ):
            # Default Server Rendering = True for Remote, False for Local
            # Can also use model='client' for full local, do not know if there is a difference
            view = plotter_ui(plotter, model='trame', default_server_rendering=False)
            ctrl.view_update = view.update
            ctrl.view_reset_camera = view.reset_camera

In [ ]:
# Load the GUI in Jupyter
await layout.ready
layout